In [1]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
 .master("spark://192.168.2.80:7077") \
 .appName("Viktor")\
 .config("spark.dynamicAllocation.enabled", True)\
 .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
 .config("spark.shuffle.service.enabled", False)\
 .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
 .config("spark.cores.max", 4)\
 .getOrCreate()

spark_context = spark_session.sparkContext


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/13 15:09:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#Import CSV file containing all songs and features.
df=spark_session.read.load("hdfs://192.168.2.80:9000/data/SongCSV.csv",format="csv", sep=",", inferSchema="true", header="true")


In [3]:
import time

start_time = time.time()

df.printSchema()

root
 |-- SongNumber: integer (nullable = true)
 |-- SongID: string (nullable = true)
 |-- AlbumID: integer (nullable = true)
 |-- AlbumName: string (nullable = true)
 |-- ArtistID: string (nullable = true)
 |-- ArtistLatitude: double (nullable = true)
 |-- ArtistLocation: string (nullable = true)
 |-- ArtistLongitude: double (nullable = true)
 |-- ArtistName: string (nullable = true)
 |-- Danceability: double (nullable = true)
 |-- Duration: double (nullable = true)
 |-- KeySignature: integer (nullable = true)
 |-- KeySignatureConfidence: double (nullable = true)
 |-- Tempo: double (nullable = true)
 |-- TimeSignature: integer (nullable = true)
 |-- TimeSignatureConfidence: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Year: integer (nullable = true)



In [4]:


#Find out the amount of songs longer then 300 seconds.
df_1 = df.withColumn('Duration', df['Duration'].cast("float"))
df_1 = df_1.filter(df['Duration'] >= 300)
print("Amount of songs longer than 300 sec: "+str(df_1.count()))





Amount of songs longer than 300 sec: 1794


In [5]:
# import time

# start_time = time.time()


df.groupBy("ArtistName").count().orderBy('count', ascending=False).show()

# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Elapsed time: {elapsed_time:.2f} seconds")


+--------------------+-----+
|          ArtistName|count|
+--------------------+-----+
|    Mario Rosenstock|   13|
|        Phil Collins|   12|
|The Jackson South...|   12|
|        Sugar Minott|   12|
|           Aerosmith|   12|
|         Mikey Dread|   11|
|Nick Cave & The B...|   11|
|         Snow Patrol|   11|
|Bill & Gloria Gai...|   11|
|         Line Renaud|   11|
|           Radiohead|   11|
|Red Hot Chili Pep...|   10|
|                 Ayo|   10|
|         White Heart|   10|
|              Selena|   10|
|       Jacques Douai|   10|
|     Pumpkin Buzzard|   10|
|Stevie Ray Vaugha...|   10|
|             Shakira|   10|
|The Del McCoury Band|   10|
+--------------------+-----+
only showing top 20 rows



In [6]:
# import time

# start_time = time.time()

from pyspark.sql.functions import avg
grouped_df=df.groupBy("ArtistName")
avg_duration_df = grouped_df.agg(avg("Duration").alias("avg_duration"))
avg_duration_df.orderBy("avg_duration", ascending=False).show()

# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Elapsed time: {elapsed_time:.2f} seconds")

+--------------------+-----------------+
|          ArtistName|     avg_duration|
+--------------------+-----------------+
|   Ministry of Sound|        1686.7522|
|        Morton Gould|       1610.00444|
|     Kartik Seshadri|       1519.28118|
|   Frederic Galliano|       1394.75546|
|The Revolutionary...|       1253.92934|
|        Small Facers|       1141.08036|
|Naseebo Lal_ Arif...|       1037.68771|
|Arthur Fiedler;Ea...|        976.56118|
|Mickey And The St...|        914.02404|
|G\xc3\xb6teborgs ...|        904.80281|
|      Henri Pousseur|884.5735860000001|
|Saarbr\xc3\xbccke...|        827.66322|
|     Kurt Tepperwein|        819.87873|
|       Guitar Shorty|        817.37098|
|          Groundhogs|       812.590565|
|Wyclef Jean featu...|        809.58649|
|The Dreamtree Pro...|         798.1971|
|       Faiz Ali Faiz|785.9021333333334|
|Herbert von Karaj...|        770.35057|
| David Mingyue Liang|        757.99465|
+--------------------+-----------------+
only showing top

In [7]:
# import time

# start_time = time.time()

task_year_df = df[df['year'] != 0]
task_year_df.groupBy("year").count().orderBy('count', ascending=False).show()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


23/03/13 15:10:22 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.2.132: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/03/13 15:10:22 WARN TaskSetManager: Lost task 0.0 in stage 11.0 (TID 8) (192.168.2.132 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


+----+-----+
|year|count|
+----+-----+
|2006|  320|
|2005|  304|
|2007|  285|
|2004|  270|
|2003|  254|
|2008|  253|
|2009|  250|
|2001|  217|
|2002|  198|
|2000|  192|
|1999|  178|
|1998|  151|
|1995|  135|
|1997|  130|
|1993|  126|
|1994|  115|
|1996|  109|
|1992|   98|
|1991|   91|
|1990|   72|
+----+-----+
only showing top 20 rows

Elapsed time: 43.71 seconds


In [8]:
# print(pd.__version__)

In [ ]:
from pyspark.sql.functions import lower, trim, regexp_replace

def print_details(rows_artist):
    for row in rows_artist.collect():
        print(f"song name: {row['Title']}")
        print(f"Album name: {row['AlbumName']}")
        print(f"Year of release: {row['Year']}")
        print("\n")
    

def want_continue():
    loop = True
    while True:
        ans = input("Want to continue looking up artist? (y/n)").lower()
        if ans == "y":
            return True
        elif ans == "n":
            loop = False
            return False
     
def artist(df):
    # preprocess lowercase 
    df = df.withColumn("ArtistName", lower(df["ArtistName"]))
    df = df.withColumn("ArtistName", regexp_replace(df["ArtistName"], " ", ""))
    select_artist = True

    while select_artist:
        user_input = input("Please enter the name of an artist: ").replace(" ", "").lower()

        # If artist exists
        if df.filter(df["ArtistName"] == user_input).count() > 0:
            rows_artist = df.filter(df["ArtistName"] == user_input)
            print_details(rows_artist)
            select_artist = want_continue()
            print("*****************")
        # if not
        else:
            print("Bad input try again")
  
artist(df)

In [ ]:
spark_session.stop()